In [1]:
import sys

print(sys.executable)

/usr/local/bin/python


In [2]:
import rospy
import torch
from A2C import A2C
import matplotlib.pyplot as plt
import numpy as np
from std_msgs.msg import Bool, Float32
from cv_bridge import CvBridge
from sensor_msgs.msg import CameraInfo
from geometry_msgs.msg import Point
from mavros_msgs.msg import State

ModuleNotFoundError: No module named 'cv_bridge'

In [ ]:
class Env:
    def __init__(self) -> None:
        rospy.init_node("train")

        self.obs = None
        self.rewards = None
        self.terminated = None
        self.ready = False
        self.bridge = CvBridge()
        self.actions = np.array(
            [
                [0, 0, 0],
                [0, 0, 1],
                [0, 0, -1],
                [0, 1, 0],
                [0, -1, 0],
                [1, 0, 0],
                [-1, 0, 0],
            ]
        )

        self.finished_sub = rospy.Subscriber(
            "/rover/finished", Bool, self.finished_callback
        )
        self.reward_sub = rospy.Subscriber("/inst_score", Float32, self.reward_callback)
        self.ready_sub = rospy.Subscriber("/drone/ready", Bool, self.ready_callback)
        self.fly_towards_pub = rospy.Publisher("/fly_towards", Point, queue_size=10)
        
        print("wait for drone ready")
        while not self.ready:
            rospy.sleep(10)

    def reset(self):
        # environment setup
        self.obs = np.zeros((1, 3, 480, 640))
        self.rewards = np.zeros((1, 1))
        self.terminated = False

    def step(self, action):
        """interact with the environment, get observation and reward"""
        self.fly_towards(direction=self.actions[action[0]])

        observation = self.observation  # 1*480*640*3
        reward = np.array([self.reward])
        terminated = [self.terminated]
        return observation, reward, terminated, False, None

    def finished_callback(self, msg):
        self.terminated = msg.data

    def reward_callback(self, msg):
        self.reward = msg.data

    def camera_info_callback(self, msg: CameraInfo):
        """Callback from camera projetion"""
        self.camera_info = msg

    def image_callback(self, msg):
        if self.camera_info is None:
            rospy.logerr("no camera info")
            return
        obs = self.bridge.imgmsg_to_cv2(msg, desired_encoding="bgr8")
        self.obs = np.array([obs])  # 480*640*3
    
    def ready_callback(self, msg):
        self.ready = msg.data


In [ ]:

envs = Env()

# environment hyperparams
n_envs = 1
n_updates = 1000
n_steps_per_update = 128
randomize_domain = False

# agent hyperparams
gamma = 0.999
lam = 0.95  # hyperparameter for GAE
ent_coef = 0.01  # coefficient for the entropy bonus (to encourage exploration)
actor_lr = 0.001
critic_lr = 0.005

# Note: the actor has a slower learning rate so that the value targets become
# more stationary and are theirfore easier to estimate for the critic

# environment setup
obs = np.zeros((1, 3, 480, 640))
rewards = np.zeros((1, 1))
actions = np.array(
    [[0, 0, 0], [0, 0, 1], [0, 0, -1], [0, 1, 0], [0, -1, 0], [1, 0, 0], [-1, 0, 0]]
)

# set the device
use_cuda = False
if use_cuda:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")

# init the agent
obs_shape = envs.obs.shape
action_shape = envs.actions.shape
agent = A2C(obs_shape, action_shape, device, critic_lr, actor_lr, n_envs)

critic_losses = []
actor_losses = []
entropies = []
return_queue = []

for sample_phase in range(n_updates):
    # we don't have to reset the envs, they just continue playing
    # until the episode is over and then reset automatically

    # reset lists that collect experiences of an episode (sample phase)
    ep_value_preds = torch.zeros(n_steps_per_update, n_envs, device=device)
    ep_rewards = torch.zeros(n_steps_per_update, n_envs, device=device)
    ep_action_log_probs = torch.zeros(n_steps_per_update, n_envs, device=device)
    masks = torch.zeros(n_steps_per_update, n_envs, device=device)

    # at the start of training reset all envs to get an initial state
    if sample_phase == 0:
        states, info = envs.reset()

    # play n steps in our parallel environments to collect data
    for step in range(n_steps_per_update):
        # select an action A_{t} using S_{t} as input for the agent
        actions, action_log_probs, state_value_preds, entropy = agent.select_action(
            states
        )

        # perform the action A_{t} in the environment to get S_{t+1} and R_{t+1}
        states, rewards, terminated, truncated, infos = envs.step(actions.cpu().numpy())

        ep_value_preds[step] = torch.squeeze(state_value_preds)
        ep_rewards[step] = torch.tensor(rewards, device=device)
        ep_action_log_probs[step] = action_log_probs

        # add a mask (for the return calculation later);
        # for each env the mask is 1 if the episode is ongoing and 0 if it is terminated (not by truncation!)
        masks[step] = torch.tensor([not term for term in terminated])

    # calculate the losses for actor and critic
    critic_loss, actor_loss = agent.get_losses(
        ep_rewards,
        ep_action_log_probs,
        ep_value_preds,
        entropy,
        masks,
        gamma,
        lam,
        ent_coef,
        device,
    )

    # update the actor and critic networks
    agent.update_parameters(critic_loss, actor_loss)

    # log the losses and entropy
    critic_losses.append(critic_loss.detach().cpu().numpy())
    actor_losses.append(actor_loss.detach().cpu().numpy())
    entropies.append(entropy.detach().mean().cpu().numpy())
    return_queue.append(np.mean(ep_rewards))

""" plot the results """

# %matplotlib inline

rolling_length = 20
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(12, 5))
fig.suptitle(
    f"Training plots for {agent.__class__.__name__} in the LunarLander-v2 environment \n \
             (n_envs={n_envs}, n_steps_per_update={n_steps_per_update}, randomize_domain={randomize_domain})"
)

# episode return
axs[0][0].set_title("Episode Returns")
episode_returns_moving_average = (
    np.convolve(
        np.array(return_queue).flatten(),
        np.ones(rolling_length),
        mode="valid",
    )
    / rolling_length
)
axs[0][0].plot(
    np.arange(len(episode_returns_moving_average)) / n_envs,
    episode_returns_moving_average,
)
axs[0][0].set_xlabel("Number of episodes")

# entropy
axs[1][0].set_title("Entropy")
entropy_moving_average = (
    np.convolve(np.array(entropies), np.ones(rolling_length), mode="valid")
    / rolling_length
)
axs[1][0].plot(entropy_moving_average)
axs[1][0].set_xlabel("Number of updates")


# critic loss
axs[0][1].set_title("Critic Loss")
critic_losses_moving_average = (
    np.convolve(
        np.array(critic_losses).flatten(), np.ones(rolling_length), mode="valid"
    )
    / rolling_length
)
axs[0][1].plot(critic_losses_moving_average)
axs[0][1].set_xlabel("Number of updates")


# actor loss
axs[1][1].set_title("Actor Loss")
actor_losses_moving_average = (
    np.convolve(np.array(actor_losses).flatten(), np.ones(rolling_length), mode="valid")
    / rolling_length
)
axs[1][1].plot(actor_losses_moving_average)
axs[1][1].set_xlabel("Number of updates")

plt.tight_layout()
plt.show()
